In [38]:
import pandas as pd
import scikit_posthocs as sp
from scipy.stats import kruskal

df = pd.read_excel("data5.xlsx", sheet_name=1, index_col='state_name', na_values='..')

# оставляем страны, по которым есть полные данные с 2003 по 2022
df = df.groupby('state_name').filter(lambda x: x['year'].min() <= 2002 and x['year'].max() >= 2022)

# удаляем чрезмерно экономически развитые страны (выбросы)
df = df.drop(['Bahrain', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'Trinidad and Tobago', 
              'United Arab Emirates', 'Chile', 'Croatia', 'Hungary', 'Poland', 'Uruguay', 'Equatorial Guinea'])

# удаляем Океанию
df = df[df['region'] != 7]

# выносим названия некоторых колонок в отдельный список для последующих манипуляций
cols = ['libdem_norm', 'property_rights', 'tax_burden', 'government_spending', 
        'business_freedom', 'monetary_freedom', 'trade_freedom', 'investment_freedom', 
        'financial_freedom', 'gov_effectiveness', 'corruption', 'pstab']

cols_1 = ['libdem_norm_diff', 'property_rights_diff', 'tax_burden_diff', 'government_spending_diff', 
        'business_freedom_diff', 'monetary_freedom_diff', 'trade_freedom_diff', 'investment_freedom_diff', 
        'financial_freedom_diff', 'gov_effectiveness_diff', 'corruption_diff', 'pstab_diff']

In [39]:
# разделяем на группы
df_11_plus = df[df.groupby('state_name')['imf_pr'].transform('sum') >= 11]
df_5_to_10 = df[(df.groupby('state_name')['imf_pr'].transform('sum') >= 5) & (df.groupby('state_name')['imf_pr'].transform('sum') <= 10)]
df_4_minus = df[df.groupby('state_name')['imf_pr'].transform('sum') <= 4]

# оставляем только 2002
df_11_plus_2002 = df_11_plus[df_11_plus['year'] == 2002]
df_5_to_10_2002 = df_5_to_10[df_5_to_10['year'] == 2002]
df_4_minus_2002 = df_4_minus[df_4_minus['year'] == 2002]

# оставляем только 2022
df_11_plus_2022 = df_11_plus[df_11_plus['year'] == 2022]
df_5_to_10_2022 = df_5_to_10[df_5_to_10['year'] == 2022]
df_4_minus_2022 = df_4_minus[df_4_minus['year'] == 2022]

In [40]:
# выводим средние значения для 2002 и 2022 в описательных целях
for col in cols:
    print(col)
    print(f'Среднее для группы 11+: {df_11_plus_2002[col].mean().round(2)} (2002); {df_11_plus_2022[col].mean().round(2)} (2022)')
    print(f'Среднее для группы 5-10: {df_5_to_10_2002[col].mean().round(2)} (2002); {df_5_to_10_2022[col].mean().round(2)} (2022)')
    print(f'Среднее для группы 4-: {df_4_minus_2002[col].mean().round(2)} (2002); {df_4_minus_2022[col].mean().round(2)} (2022)')
    print()

libdem_norm
Среднее для группы 12+: 32.48 (2002); 36.43 (2022)
Среднее для группы 5-11: 43.31 (2002); 40.42 (2022)
Среднее для группы 4-: 40.46 (2002); 32.87 (2022)

property_rights
Среднее для группы 12+: 38.22 (2002); 42.23 (2022)
Среднее для группы 5-11: 38.67 (2002); 45.22 (2022)
Среднее для группы 4-: 37.41 (2002); 45.73 (2022)

tax_burden
Среднее для группы 12+: 69.45 (2002); 80.61 (2022)
Среднее для группы 5-11: 73.95 (2002); 80.41 (2022)
Среднее для группы 4-: 72.37 (2002); 81.1 (2022)

government_spending
Среднее для группы 12+: 77.14 (2002); 80.3 (2022)
Среднее для группы 5-11: 75.57 (2002); 73.25 (2022)
Среднее для группы 4-: 77.42 (2002); 75.96 (2022)

business_freedom
Среднее для группы 12+: 56.0 (2002); 50.86 (2022)
Среднее для группы 5-11: 57.5 (2002); 57.85 (2022)
Среднее для группы 4-: 57.78 (2002); 59.1 (2022)

monetary_freedom
Среднее для группы 12+: 74.52 (2002); 74.44 (2022)
Среднее для группы 5-11: 74.52 (2002); 73.89 (2022)
Среднее для группы 4-: 68.15 (2002); 67

In [41]:
# выявляем возможные различия в развитии институтов для трёх рассматриваемых групп в 2002 г.
for col in cols:
    data = [df_11_plus_2002[col], df_5_to_10_2002[col], df_4_minus_2002[col]]
    stat, p_value = kruskal(*data)
    print(col)
    print(f"Kruskal-Wallis H-statistic: {stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if p_value < 0.05:
        print()
        print("Есть статистически значимые различия, выполняем тест Данна:")
        print(sp.posthoc_dunn(data, p_adjust='holm'))
    else:
        print('Нет различий между группами')
    print()

libdem_norm
Kruskal-Wallis H-statistic: 2.9767
P-value: 0.2258
Нет различий между группами

property_rights
Kruskal-Wallis H-statistic: 0.2588
P-value: 0.8786
Нет различий между группами

tax_burden
Kruskal-Wallis H-statistic: 1.9520
P-value: 0.3768
Нет различий между группами

government_spending
Kruskal-Wallis H-statistic: 0.1639
P-value: 0.9213
Нет различий между группами

business_freedom
Kruskal-Wallis H-statistic: 0.8284
P-value: 0.6609
Нет различий между группами

monetary_freedom
Kruskal-Wallis H-statistic: 1.8896
P-value: 0.3888
Нет различий между группами

trade_freedom
Kruskal-Wallis H-statistic: 0.1379
P-value: 0.9334
Нет различий между группами

investment_freedom
Kruskal-Wallis H-statistic: 5.1502
P-value: 0.0761
Нет различий между группами

financial_freedom
Kruskal-Wallis H-statistic: 2.9662
P-value: 0.2269
Нет различий между группами

gov_effectiveness
Kruskal-Wallis H-statistic: 5.8789
P-value: 0.0529
Нет различий между группами

corruption
Kruskal-Wallis H-statistic:

In [42]:
# выявляем возможные различия в развитии институтов для трёх рассматриваемых групп в 2022 г.
for col in cols:
    data = [df_11_plus_2022[col], df_5_to_10_2022[col], df_4_minus_2022[col]]
    stat, p_value = kruskal(*data)
    print(col)
    print(f"Kruskal-Wallis H-statistic: {stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if p_value < 0.05:
        print()
        print("Есть статистически значимые различия, выполняем тест Данна:")
        print(sp.posthoc_dunn(data, p_adjust='holm'))
    else:
        print('Нет различий между группами')
    print()

libdem_norm
Kruskal-Wallis H-statistic: 2.0856
P-value: 0.3525
Нет различий между группами

property_rights
Kruskal-Wallis H-statistic: 0.9398
P-value: 0.6251
Нет различий между группами

tax_burden
Kruskal-Wallis H-statistic: 0.4555
P-value: 0.7963
Нет различий между группами

government_spending
Kruskal-Wallis H-statistic: 4.1926
P-value: 0.1229
Нет различий между группами

business_freedom
Kruskal-Wallis H-statistic: 6.7696
P-value: 0.0339

Есть статистически значимые различия, выполняем тест Данна:
          1         2         3
1  1.000000  0.175579  0.032682
2  0.175579  1.000000  0.376458
3  0.032682  0.376458  1.000000

monetary_freedom
Kruskal-Wallis H-statistic: 1.3367
P-value: 0.5126
Нет различий между группами

trade_freedom
Kruskal-Wallis H-statistic: 4.1634
P-value: 0.1247
Нет различий между группами

investment_freedom
Kruskal-Wallis H-statistic: 2.5335
P-value: 0.2817
Нет различий между группами

financial_freedom
Kruskal-Wallis H-statistic: 0.1377
P-value: 0.9335
Нет 

In [43]:
# создаём датафреймы для сравнения динамики
df_11_plus_ = df_11_plus[df_11_plus['year'] > 2002]
df_5_to_10_ = df_5_to_10[df_5_to_10['year'] > 2002]
df_4_minus_ = df_4_minus[df_4_minus['year'] > 2002]

In [44]:
# сравниваем средние изменения институциональных переменных у трёх рассматриваемых групп стран за период 2003-2022
for col in cols_1:
    data = [df_11_plus_[col], df_5_to_10_[col], df_4_minus_[col]]
    stat, p_value = kruskal(*data)
    print(col)
    print(f"Kruskal-Wallis H-statistic: {stat:.4f}")
    print(f"P-value: {p_value:.4f}")
    if p_value < 0.05:
        print()
        print("Есть статистически значимые различия, выполняем тест Данна:")
        print(sp.posthoc_dunn(data, p_adjust='holm'))
    else:
        print('Нет различий между группами')
    print()

libdem_norm_diff
Kruskal-Wallis H-statistic: 5.8151
P-value: 0.0546
Нет различий между группами

property_rights_diff
Kruskal-Wallis H-statistic: 3.2914
P-value: 0.1929
Нет различий между группами

tax_burden_diff
Kruskal-Wallis H-statistic: 9.2984
P-value: 0.0096

Есть статистически значимые различия, выполняем тест Данна:
          1         2         3
1  1.000000  0.024033  0.773165
2  0.024033  1.000000  0.018834
3  0.773165  0.018834  1.000000

government_spending_diff
Kruskal-Wallis H-statistic: 0.6773
P-value: 0.7127
Нет различий между группами

business_freedom_diff
Kruskal-Wallis H-statistic: 3.1161
P-value: 0.2105
Нет различий между группами

monetary_freedom_diff
Kruskal-Wallis H-statistic: 0.1860
P-value: 0.9112
Нет различий между группами

trade_freedom_diff
Kruskal-Wallis H-statistic: 0.0613
P-value: 0.9698
Нет различий между группами

investment_freedom_diff
Kruskal-Wallis H-statistic: 1.2623
P-value: 0.5320
Нет различий между группами

financial_freedom_diff
Kruskal-Wa